In [32]:
using DataFrames
using CSV
using ScatteredInterpolation
using CairoMakie
using AlgebraOfGraphics


function find_missing_columns(df::DataFrame)
    return [col for col in names(df) if any(ismissing.(df[!, col]))]
end

function remove_missing_columns!(df::DataFrame)
    missing_cols = find_missing_columns(df)
    select!(df, Not(missing_cols))
    return df
end

function load_isochrone_data(filepath::String)
    df = DataFrame(CSV.File(filepath, delim=' ', ignorerepeated=true, comment="#", silencewarnings=true))
    remove_missing_columns!(df)
    return df
end

function add_evolution_phase!(df::DataFrame)
    df.evol = zeros(nrow(df))
    for label in unique(df.label)
        idx = df.label .== label
        n_points = sum(idx)
        df[idx, :evol] = label .+ range(0, 1 - 1/n_points, length=n_points)
    end
    return df
end

function find_interval_uniform(A::Vector{T}, x::T) where T<:Real
    if x < first(A) || x > last(A)
        error("x está fuera del rango de A. x debe estar en [$(first(A)), $(last(A))].")
    end

    if x == last(A)
        return length(A) - 1
    end

    step = (last(A) - first(A)) / (length(A) - 1)
    i = floor(Int, (x - first(A)) / step) + 1
    return i
end

function find_nearest_isochrones(df::DataFrame, log_age::Real, metal::Real)
    age_vals = sort(unique(df.logAge))
    metal_vals = sort(unique(df.MH))

    age_idx = find_interval_uniform(age_vals, log_age)
    metal_idx = find_interval_uniform(metal_vals, metal)

    age_nodes = age_vals[age_idx:age_idx+1]
    metal_nodes = metal_vals[metal_idx:metal_idx+1]

    return df[(df.logAge .∈ [age_nodes]) .& (df.MH .∈ [metal_nodes]), :]
end

function interpolate_isochrone(df::DataFrame, target_age::Real, target_metallicity::Real)
    log_age = log10(target_age)
    nearest = find_nearest_isochrones(df, log_age, target_metallicity)
    add_evolution_phase!(nearest)

    points = Matrix(nearest[:, [:logAge, :MH, :evol]])
    mag_cols = setdiff(names(nearest), [:logAge, :MH, :label, :evol])

    max_label = maximum(nearest.label)
    @show max_label
    evol_points = range(0, max_label + 0.99, length=1_000*(max_label+1))

    targets = hcat(
        fill(log_age, length(evol_points)),
        fill(target_metallicity, length(evol_points)),
        collect(evol_points)
    )

    result = DataFrame(
        evol = evol_points,
        logAge = fill(log_age, length(evol_points)),
        MH = fill(target_metallicity, length(evol_points)),
        label = floor.(Int, evol_points)
    )

    for col in mag_cols
        itp = interpolate(Multiquadratic(), points', nearest[:, col])
        result[!, col] = evaluate(itp, targets')
    end

    return dropmissing(result)
end

"""Plot single isochrone using AlgebraOfGraphics."""
function plot_isochrone_cmd(df::DataFrame, telescope::Symbol, file::String)
    size_inches = (3 * 3, 3 * 3)
    size_pt = 72 .* size_inches
    fig = Figure(size = size_pt, fontsize = 30)

    if telescope == :Gaia
        df.color = df."BP-RP"
        plt = data(df) * mapping(:color => L"BP-RP", :Gaia_G_EDR3 => L"G") * visual(Lines)
        draw!(fig, plt, axis = (; yreversed = true))
    elseif telescope == :Subaru
        df.color = df.gmag - df.rmag
        plt = data(df) * mapping(:color => L"g-r", :gmag => L"g", color=:label) * visual(Lines, colormap=)
        draw!(fig, plt, axis = (; yreversed = true), theme = Theme(palette = (color = Makie.tab10,)))
    else
        error("Telescope $telescope not available for CMD plot.")
    end

    save(file, fig, pt_per_unit = 1)
    return fig
end

"""Example use with Subaru data."""
function example_interpolate_isochrone()
    filter = "hsc"
    file_artif = "../artifacts/isochrones/parsec/$filter/family_MH_-2.2_0.5_logAge_9.2_10.3.dat"
    df_artif = load_isochrone_data(file_artif)
    file_plot = "../plots/isochrone_$filter.pdf"

    target_age = 10.0^9.8
    target_metallicity = 0.0

    isochrone_df = interpolate_isochrone(df_artif, target_age, target_metallicity)
    fig = plot_isochrone_cmd(isochrone_df, :Subaru, file_plot)
    println("Isócrona interpolada y graficada para Subaru.")

    return isochrone_df
end


Base.Meta.ParseError: ParseError:
# Error @ /home/mmestre/.julia/dev/IntrospectiveStreams/notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W0sZmlsZQ==.jl:106:106
        df.color = df.gmag - df.rmag
        plt = data(df) * mapping(:color => L"g-r", :gmag => L"g", color=:label) * visual(Lines, colormap=)
#                                                                                                        ╙ ── unexpected `)`

In [33]:
example_interpolate_isochrone()

max_label = 9
Isócrona interpolada y graficada para Subaru.


Row,evol,logAge,MH,label,Zini,Mini,int_IMF,Mass,logL,logTe,logg,McoreTP,C_O,period0,period1,period2,period3,period4,pmode,Mloss,tau1m,X,Y,Xc,Xn,Xo,Cexcess,Z,\t,mbolmag,gmag,rmag,imag,zmag,ymag,NB816mag,color
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.214995,10.0427,-0.0149314,184.637,0.0146137,1.66045e5,4.60857e5,1.60465e5,1.81817e6,-1.77111e5,-2.20628e6,1.16359e5,-12834.1,4.19611e8,1.53277e8,9.2581e7,7.48883e7,6.11438e7,-1.2013e7,-1.13175,6029.53,-23959.0,23226.8,103.483,139.275,418.99,-1.02283,999.83,-4.54878e6,3.21547e6,3.08724e6,-1.11372e6,-2.60776e6,-3.70524e6,-2.87193e6,-3.44442e6,4.20096e6
2,0.215238,10.042,-0.0149302,184.258,0.0146126,1.65675e5,4.59873e5,160104.0,1.81428e6,-1.76763e5,-2.20169e6,1.16126e5,-12811.3,4.18758e8,1.52922e8,9.23654e7,7.47139e7,6.10014e7,-1.19887e7,-1.12966,6021.15,-23911.1,23180.2,103.279,138.988,418.147,-1.02275,997.986,-4.53904e6,3.20961e6,3.08131e6,-1.11069e6,-2.6017e6,-3.69701e6,-2.86521e6,-3.43662e6,4.19199e6
3,0.215482,10.0412,-0.0149291,183.879,0.0146116,1.65305e5,4.58889e5,1.59743e5,1.81038e6,-1.76415e5,-2.19709e6,1.15895e5,-12788.5,4.17906e8,1.52568e8,9.21499e7,7.45398e7,6.08594e7,-1.19643e7,-1.12756,6012.55,-23863.2,23133.6,103.075,138.701,417.305,-1.02268,996.143,-4.52929e6,3.20376e6,3.07538e6,-1.10765e6,-2.59565e6,-3.68879e6,-2.8585e6,-3.42882e6,4.18303e6
4,0.215728,10.0404,-0.014928,183.499,0.0146105,1.64933e5,4.57903e5,1.59381e5,1.80648e6,-1.76067e5,-2.19249e6,1.1566e5,-12765.6,4.17052e8,1.52213e8,9.19335e7,7.43651e7,6.07165e7,-1.19399e7,-1.12547,6004.13,-23815.1,23086.9,102.871,138.414,416.462,-1.0226,994.296,-4.51951e6,3.1979e6,3.06944e6,-1.10461e6,-2.58957e6,-3.68054e6,-2.85177e6,-3.42099e6,4.17404e6
5,0.215974,10.0396,-0.0149268,183.119,0.0146094,1.64563e5,4.56919e5,1.59019e5,1.80258e6,-1.75719e5,-2.18789e6,1.15429e5,-12742.8,4.162e8,1.51858e8,9.17179e7,7.41908e7,6.05743e7,-1.19156e7,-1.12337,5995.71,-23767.1,23040.2,102.667,138.127,415.618,-1.02253,992.45,-4.50976e6,3.19204e6,3.0635e6,-1.10157e6,-2.58352e6,-3.67231e6,-2.84506e6,-3.41319e6,4.16507e6
6,0.216223,10.0388,-0.0149257,182.739,0.0146083,1.64192e5,4.55932e5,1.58657e5,1.79867e6,-1.7537e5,-2.18328e6,1.15196e5,-12719.9,4.15345e8,1.51503e8,9.15018e7,7.40161e7,6.04317e7,-1.18912e7,-1.12127,5987.03,-23719.1,22993.4,102.462,137.84,414.774,-1.02245,990.601,-4.49999e6,3.18616e6,3.05755e6,-1.09853e6,-2.57744e6,-3.66407e6,-2.83832e6,-3.40537e6,4.15608e6
7,0.216473,10.038,-0.0149246,182.358,0.0146072,1.63819e5,4.54942e5,1.58294e5,1.79476e6,-1.7502e5,-2.17866e6,1.14962e5,-12697.0,4.14489e8,1.51147e8,9.12847e7,7.38407e7,6.02885e7,-1.18667e7,-1.11917,5978.74,-23670.9,22946.5,102.257,137.552,413.927,-1.02238,988.751,-4.49019e6,3.18029e6,3.0516e6,-1.09547e6,-2.57136e6,-3.6558e6,-2.83157e6,-3.39752e6,4.14707e6
8,0.216724,10.0373,-0.0149234,181.979,0.0146062,1.63449e5,4.53958e5,1.57933e5,1.79086e6,-1.74672e5,-2.17406e6,1.14729e5,-12674.1,4.13636e8,1.50792e8,9.10693e7,7.36667e7,6.01462e7,-1.18423e7,-1.11707,5970.16,-23622.9,22899.9,102.053,137.265,413.084,-1.0223,986.903,-4.48044e6,3.17442e6,3.04565e6,-1.09244e6,-2.5653e6,-3.64756e6,-2.82486e6,-3.38971e6,4.1381e6
9,0.216977,10.0365,-0.0149223,181.597,0.0146051,1.63076e5,4.52969e5,1.57569e5,1.78694e6,-1.74322e5,-2.16944e6,114495.0,-12651.2,4.12779e8,1.50436e8,9.08522e7,7.34913e7,6.00031e7,-1.18179e7,-1.11497,5961.7,-23574.7,22853.0,101.848,136.977,412.237,-1.02222,985.05,-4.47063e6,3.16854e6,3.0397e6,-1.08938e6,-2.5592e6,-3.63929e6,-2.81809e6,-3.38186e6,4.12908e6
